In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# ==========================================
# 1. LOAD DATA & PREPROCESSING
# ==========================================
print("Sedang memproses data...")
df = pd.read_csv('Latest 2025 movies Datasets.csv')

# Membersihkan data
df = df.dropna(subset=['release_date', 'vote_average', 'popularity', 'vote_count'])
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df = df.dropna(subset=['release_date'])

# Feature Engineering
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

# Encoding Bahasa
le = LabelEncoder()
df['original_language_encoded'] = le.fit_transform(df['original_language'])

# Fitur & Target
features = ['popularity', 'vote_count', 'release_year', 'release_month', 'original_language_encoded']
X = df[features]
y = df['vote_average']

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- SCALING (Wajib untuk KNN) ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Kita juga perlu scale X full untuk Cross Validation nanti
X_scaled = scaler.transform(X)

# ==========================================
# 2. TRAINING MODEL (KNN)
# ==========================================
print("Sedang melatih model KNN (15 Tetangga)...")
model = KNeighborsRegressor(n_neighbors=15)
model.fit(X_train_scaled, y_train)

# ==========================================
# 3. EVALUASI LENGKAP
# ==========================================
print("\n" + "="*40)
print("HASIL EVALUASI (KNN)")
print("="*40)

# A. Single Split Test
y_pred = model.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"1. Single Split Test:")
print(f"   - R2 Score (Akurasi) : {r2:.4f} ({r2*100:.2f}%)")
print(f"   - RMSE (Error Kuadrat): {rmse:.4f}")
print(f"   - MAE (Rata-rata Error): {mae:.4f} poin")

# B. Cross Validation (5-Fold)
print(f"\n2. Cross Validation (5-Fold):")
# Kita gunakan X_scaled (data penuh yang sudah di-scale)
cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='r2')

print(f"   - Skor per tes : {cv_scores}")
print(f"   - Rata-rata R2 : {cv_scores.mean():.4f}")
print(f"   - Kestabilan   : +/- {cv_scores.std():.4f}")

# Contoh Prediksi
print("\n" + "="*40)
print(f"Contoh Prediksi: Rating Asli {y_test.iloc[0]} | Prediksi KNN {y_pred[0]:.2f}")

Sedang memproses data...
Sedang melatih model KNN (15 Tetangga)...

HASIL EVALUASI (KNN)
1. Single Split Test:
   - R2 Score (Akurasi) : 0.2060 (20.60%)
   - RMSE (Error Kuadrat): 2.1397
   - MAE (Rata-rata Error): 1.5157 poin

2. Cross Validation (5-Fold):
   - Skor per tes : [0.10580954 0.07685483 0.12809619 0.14718973 0.0873096 ]
   - Rata-rata R2 : 0.1091
   - Kestabilan   : +/- 0.0258

Contoh Prediksi: Rating Asli 7.0 | Prediksi KNN 5.32
